# DevOps for SailPoint IdentityNow
## SailPoint Navigate Developer On Demand Session - August 2021
[Developer Day](https://navigate.sailpoint.com/?utm_source=linkedin.com&utm_medium=social&utm_campaign=navigate&utm_content=linkedin_event)

Install the latest version of the SailPoint IdentityNow PowerShell Module from the PowerShell Gallery. Once you have the PowerShell Module installed you can comment out the first line below.

**install-module SailPointIdentityNow -force**

Import the SailPoint IdentityNow Module to the local Jupyter environment.

In [ ]:
Import-Module SailPointIdentityNow -RequiredVersion 1.1.5

In [ ]:
get-module

In [ ]:
Get-Command -Module SailPointIdentityNow | Sort-Object Name | Format-Table Name -Autosize

# Configure Credentials for the IdentityNow PowerShell Module
Create a SailPoint IdentityNow PowerShell Module Profile for your IdentityNow Organisation with your Admin Account and associated Personal Access Token. 


Your Personal Access Token is created under your profile in the IdentityNow Portal. Select your name in the top right hand corner of the portal. Select Preferences


![](./images/CreateNewPersonalAccessToken1.png)

Select **Personal Access Tokens** then **New Token**

![](./images/CreateNewPersonalAccessToken2.png)

Provide a name/reason/description for the Personal Access Token then select **Create Token**.

Copy the ClientID and Secret.  

![](./images/CreateNewPersonalAccessToken3.png)

## Set the IdentityNow Organisation
This is the IdentityNow Org that we will be orchestrating with PowerShell

In [ ]:
$orgName = "yourOrgName"
Get-IdentityNowOrg 

In [ ]:
$adminUSR = "yourAdminName"
$adminPWD = 'yourAdminPassword'
$adminCreds = [pscredential]::new($adminUSR, ($adminPWD | ConvertTo-SecureString -AsPlainText -Force))

# IdentityNow Personal Access Token as generated through the IdentityNow Portal and your personal identity profile preferences
$patClientID = '8489cbb6010..yourPATClientID..422679713'
$patClientSecret = 'eb72f2a94fe8b..yourPATSecret..e92120d196'
$patCreds = [pscredential]::new("$($patClientID)", ($patClientSecret | ConvertTo-SecureString -AsPlainText -Force))

Set-IdentityNowCredential -AdminCredential $adminCreds -PersonalAccessToken $patCreds

## Validate your IdentityNow credentials against the IdentityNow Organisation
**Note** In this configuration I have historical credentials from earlier versions of the module. From v1.1.4 moving forward you only require a v3/Personal Access Token as configured above. 

In [ ]:
Test-IdentityNowCredentials

## Save Credentials into your IdentityNow Configuration Profile 
With the configuration set and validated save it securely to the local profile. Make it default so it is the credentials auto loaded with the module (where you have multiple IdentityNow Org configs).  

In [ ]:
Save-IdentityNowConfiguration -default
Get-IdentityNowOrg

# Create a New Flat File Source
With our credentials and organisation configured and validated let's get a list of all the 'Sources' and look at the default 'Identity Admins' source. 

In [ ]:
$sources = Get-IdentityNowSource
$sources.count

In [ ]:
$flatFileSources = $sources | where-object {$_.sourceType -eq 'DELIMITED_FILE'} | select-object 
$sources | where-object {$_.sourceType -eq 'DELIMITED_FILE'} | select-object -First 1

## New Flat File Source using info for configuration from IdentityNow Admins

In [ ]:
$newFlatFileSource = New-IdentityNowSource -name 'External Users' -description 'External Users' -connectorname 'Delimited File' -sourcetype 'DELIMITED_FILE' 
$newFlatFileSource

## Add additional attributes to the Flat File Source Schema
Add 'company', 'jobTitle', 'state', 'fullname'

In [ ]:
New-IdentityNowSourceAccountSchemaAttribute -sourceID $newFlatFileSource.id -name 'company' -description 'External Organisation' -type 'string' 
New-IdentityNowSourceAccountSchemaAttribute -sourceID $newFlatFileSource.id -name 'jobTitle' -description 'Job Title' -type 'string' 
New-IdentityNowSourceAccountSchemaAttribute -sourceID $newFlatFileSource.id -name 'state' -description 'State' -type 'string' 
New-IdentityNowSourceAccountSchemaAttribute -sourceID $newFlatFileSource.id -name 'displayName' -description 'Display Name' -type 'string' 


# Create a new Identity Profile
Now that we have a new Source for External Identities, let's create an Identity Profile for these External Identity people.

We will pass in the ID of the new Flat File Source we created above. 

In [ ]:
$newIdentityProfile = New-IdentityNowProfile -Name Externals -SourceID $newFlatFileSource.id 

## Update the Mappings on the Identity Profile to bring through our attributes
We can now configure attributes from our Flat File Source to map to our Identity Profile. 

We will update the Identity Profile for uid, team, region, jobTitle, displayName, firstname, lastname and email. 

In [ ]:
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute uid -sourceType Standard -source "$($newFlatFileSource.name):name" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute team -sourceType Standard -source "$($newFlatFileSource.name):company" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute region -sourceType Standard -source "$($newFlatFileSource.name):state" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute jobTitle -sourceType Standard -source "$($newFlatFileSource.name):jobTitle" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute displayName -sourceType Standard -source "$($newFlatFileSource.name):displayName" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute firstname -sourceType Standard -source "$($newFlatFileSource.name):givenName" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute lastname -sourceType Standard -source "$($newFlatFileSource.name):familyName" | out-null 
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute email -sourceType Standard -source "$($newFlatFileSource.name):e-mail" | out-null 


## Update IdentityNow Profile

With the mapping configuration updated, we need to refresh the Identity Profile. 

In [ ]:
Start-IdentityNowProfileUserRefresh -id $newIdentityProfile.id

## Import Users for creation on the Flat File Source
Reference [Example Dataset](https://blog.darrenjrobinson.com/quickly-generating-a-dataset-of-fictitious-users-using-randomised-real-data-and-powershell/)

In this dataset I have the details for 20 Users. 

In [ ]:
$userDataset = Import-Clixml .\datasets\FlatFileSourceUsersDataset-20.xml
$userDataset.count 
$userDataset[11]

## Import Entitlements for creation on the Flat File Source

On Flat File Sources we have input files for users, the list of entitlments for a source, then each user and the entitlements they have. 
The following is the list of entitlements on the Source. 

In [ ]:
$entitlementData = import-csv ./datasets/sourceEntitlements.csv 
$entitlementData[11]

## Create Users on the Flat File Source
Using the list of users we imported from a local file above we will iterate through them and create them on our new source. 


In [ ]:
foreach ($user in $userDataset){
    $account = $null 
    $account = @{"id"    = $user.Username; 
            "name"        = $user.Username; 
            "givenName"   = $user.Givenname;             
            "familyName"  = $user.Surname; 
            "displayName" = $user.Fullname; 
            "e-mail"      = $user.email;
            "state"       = $user.State;
            "company"     = $user.Company;
            "jobTitle"    = $user.Title;
        }
    "Creating user $($user.Fullname)"
    New-IdentityNowUserSourceAccount -source $newFlatFileSource.id -account ($account | convertto-json)
}    

## Check on the Identity Processing 
We can then check to see IdentityNow processing the new users. 

In [ ]:
Get-IdentityNowActiveJobs

## Create the Entitlements on the Flat File Source
Using the entitlments file for the source we imported above, we can create them on the Source. 

In [ ]:
# $entitlementData = import-csv ./datasets/sourceEntitlements.csv 
New-IdentityNowSourceEntitlements -entitlements $entitlementData -sourceID $newFlatFileSource.id 

## Import Users Entitlements for creation on Flat File Source
Finally we can import the file we have which lists each users entitlements. When a user has multiple entitlements, there will be a row in the dataset for each entitlement a user has. 

In [ ]:
$entitlementsDataSet = Import-Clixml .\datasets\FlatFileSourceUsersEntitlementDataset-2.xml
$entitlementsDataSet.count
$entitlementsDataSet[2]
$entitlementsDataSet[3]

## Import Users Entitlements
First get all the new users we've just created

In [ ]:
$flatFileSourceUserAccounts = Get-IdentityNowSourceAccounts -sourceID $newFlatFileSource.id 

### Read in the entitlements and update the User Object for them. 
Updates for Entitlements on a Flat File Source is an update per user per entitlement

In [ ]:
$entitlementsDataSet[11]

In [ ]:
foreach ($entitlement in $entitlementsDataSet){
    $userObj = $null 
    $userObj = $flatFileSourceUserAccounts | where-object {$_.accountID -eq $entitlement.id} | select-object
    Update-IdentityNowUserSourceAccount -account $userObj.id -update ($entitlement | convertto-json)
    "Updating $($entitlement.displayName) entitlements"
}


## Update an Account
We can now look at a source and retrieve a users. We will use the ID of the source we created above to return all the users on that source. 

Then we can extract and particular user from the source. 

In [ ]:
$externalUsers = Get-IdentityNowSourceAccounts -sourceID $newFlatFileSource.id
$user = $externalUsers | where-object {$_.accountId -eq 'Breanna.Mccreadie'} | select-object
$user 

## Add another attribute to the Flat File Source then update the user above
With our base records created on the source for our users, we may want to add another attribute to the Source. 

Let's add a new attribute named 'Country'. 

In [ ]:
New-IdentityNowSourceAccountSchemaAttribute -sourceID $newFlatFileSource.id -name 'country' -description 'Country' -type 'string' 

### Update the user we retrieved above to populate their country with the value 'Australia'

In [ ]:
$update = @{
    "country" = "Australia"
} 

In [ ]:
Update-IdentityNowUserSourceAccount -account $user.id -update ($update | ConvertTo-Json)

## Add Country as an IdentityProfile Attribute
First add a mapping for the Country attribute from the Source to the Identity Profile

In [ ]:
Update-IdentityNowProfileMapping -id $newIdentityProfile.id -IdentityAttribute country -sourceType Standard -source "$($newFlatFileSource.name):country"

## Refresh the Identity Profile for the new attribute

In [ ]:
Start-IdentityNowProfileUserRefresh -id $newIdentityProfile.id

### Look at the Active Jobs processing the changes

In [ ]:
Get-IdentityNowActiveJobs

## Make the 'country' attribute available for correlation rules
Add an attribute into the Identity Attributes List that can be used in Correlation Rules.

This makes the attribute searchable and avaialble for correlation rules.

In [ ]:
Get-IdentityNowIdentityAttribute -attribute country

In [ ]:
Update-IdentityNowIdentityAttribute -attribute country

## Remove a user

To remove a users from a flat file source we need to retrieve the ID of their source account user object. 

In [ ]:
$externalUsers = Get-IdentityNowSourceAccounts -sourceID $newFlatFileSource.id
$removeUser = $externalUsers | where-object {$_.accountId -eq 'Breanna.Mccreadie'} | select-object
$removeUser 

In [ ]:
Remove-IdentityNowUserSourceAccount -account $removeUser.id

# Create Access Profiles for all the Entitlements

Create an Access Profile that encompasses all the entitlements on an Source.  

**This isn't really a real world example**

### Get Owner for Access Profile

In [ ]:
$owner = Search-IdentityNowUserProfile -query "darren.robinson"
$owner

### Get Source for Access Profile

In [ ]:
$sources = Get-IdentityNowSource 
$mySource = $sources | Select-Object | Where-Object {$_.name -like '*External Users*'}
$mySource

### Get Source Entitlements
**Again not a real world scenario** but shows that we can quickly create and Access Profile with 120+ entitlements. 

We get all the entitlements available on the Source and we'll add them to a new Access Profile we will create. 

In [ ]:
# Entitlements
$e = Search-IdentityNowEntitlements -query "source.id:$($mySource.externalId)"


In [ ]:
# Access Profile Details
$accessProfile = @{}
$accessProfile.add("name", "$($mySource.name) Access Profile")
$accessProfile.add("description", "$($mySource.name) Access Profile")
$accessProfile.add("sourceId", $mySource.id)
$accessProfile.add("ownerId", $owner.id)

$accessProfile

## Configure the Access Profile 

Configure the Access Profile for Manager Approval with comments required for Request and Deny workflows. 

In [ ]:
# Access Profile Entitlements
$entitlements = @()
ForEach($i in $e) {$entitlements += $i.id}
$entitlementsToAdd = @{"entitlements" = $entitlements}
$accessProfile.add("entitlements", $entitlementsToAdd.entitlements)

# Access Profile Type
$accessProfile.add("approvalSchemes", "manager")
$accessProfile.add("requestCommentsRequired", $true)
$accessProfile.add("deniedCommentsRequired", $true)

$accessProfile

### Create the Access Profile

With the configuration created we can create the Access Profile

In [ ]:
New-IdentityNowAccessProfile -profile ($accessProfile | convertto-json)

# Create a Certification Campaign for our new Source

We will create a Source Owner based Certification Campaign for our new Flat File Source.


## Create a Campaign Filter

We will create an inclusion campaign filter for the source.

In [ ]:
$criteria = @{}
$criteria.Add("operation", "EQUALS")
$criteria.Add("property", "application")
$criteria.Add("sourceName", "")
$criteria.Add("type", "SOURCE")
$criteria.Add("value", "$($newFlatFileSource.name)")
$jsonCriteria = $criteria | convertto-json

$campaignFilter = @{}
$campaignFilter.Add("mode", "INCLUSION")
$campaignFilter.Add("name", "Externals Source Filter")
$campaignFilter.Add("description", "Source Campaign Filter - Externals")
$campaignFilter.Add("criteriaList", $jsonCriteria )


In [ ]:
$campaignFilter
$campaignFilter.criteriaList                   

In [ ]:
$campaignFilter = Invoke-IdentityNowRequest -method Post `
-uri "https://$($orgName).api.identitynow.com/cc/api/campaignFilter/create" `
-headers Headersv3_JSON `
-body ($campaignFilter | ConvertTo-Json)

## Create our Campaign

Create our Source Owner based campaign in the Australian East Timezone specifying the criteria (don't auto revoke, naming and campaign end date). 

In [ ]:
# Create Campaign
$campaignOptions = @{ }
$campaignOptions.Add("type", "SourceOwner")
$campaignOptions.Add("timeZone", "GMT+1000")
$campaignOptions.Add("name", "July 2021 $($newFlatFileSource.name) Source Entitlements Campaign")
$campaignOptions.Add("allowAutoRevoke", $false)
$campaignOptions.Add("deadline", "2021-08-01")
$campaignOptions.Add("description", "July 2021 $($newFlatFileSource.name) Source Entitlements Campaign")
$campaignOptions.Add("disableEmail", $true)
$campaignOptions.Add("identityIdList", @())
$campaignOptions.Add("sourceIds", "$($newFlatFileSource.id)")
$campaignOptions.Add("campaignFilterId", "$($campaignFilter.id)")
$campaignBody = $campaignOptions | ConvertTo-Json

In [ ]:
$campaignBody 

In [ ]:
$newCertCampaign = New-IdentityNowCertCampaign -start $false -campaign $campaignBody 
$newCertCampaign

## Start our Campaign
When we created the campaign above we used -start $false

This means it was created and sits as Preview. 

Now let's start the campaign

In [ ]:
Start-IdentityNowCertCampaign -campaignID $newCertCampaign.id -timezone "GMT+10:00"

## Campaign Reports
Get/Generate the reports for the campaign we just started. 

In [ ]:
Get-IdentityNowCertCampaign -campaignID $newCertCampaign.id

In [ ]:
$reports = Get-IdentityNowCertCampaignReport -campaignID $newCertCampaign.id 
$reports

In [ ]:
$reports[0].Report

In [ ]:
Get-IdentityNowCertCampaignReport -period 30 -completed $false 

## Search Authentications

In [ ]:
$query = @{query = 'technicalName:AUTHENTICATION_REQUEST_PASSED*'; type = 'AUTH'}
$queryFilter = @{query = $query}
Search-IdentityNowEvents -filter ($queryFilter | convertto-json) -searchLimit 100

## Applications
Get the default configured Applications (using the -org switch)

In [ ]:
$applicationsList = Get-IdentityNowApplication -org $true
$appNames = $applicationsList | select-object -Property name 

In [ ]:
$applicationsList[11]

In [ ]:
$appNames.count
